# Messari bypass paywaller

In [1]:
import requests
import os
import json
# import xmltojson
# import BeautifulSoup4 as bs

In [69]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/SunnyNie/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/4n/h7p57sb17sn7vtrtjz3m1xhr0000gn/T/ipykernel_24429/27291130.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [70]:
url = 'https://messari.io/article/the-evolution-sushiswap-to-sushi-com'
url = 'https://messari.io/article/cosmos-ibc-unlocking-the-blockchain-multiverse'

In [71]:
from selenium import webdriver

# browser = webdriver.Chrome()
driver.get(url)

In [74]:
from bs4 import BeautifulSoup
# from selenium import webdriver

# driver = webdriver.Chrome()
# html_source_code = driver.execute_script("return document.body.innerHTML;")
html_soup: BeautifulSoup = BeautifulSoup(driver.page_source, 'html.parser')

In [75]:
main = html_soup.find('div', attrs={'class': 'Messari-paywall'}).next()

In [76]:
import requests
import shutil

In [77]:
import os, glob

In [78]:
import pypandoc
from datetime import datetime


In [79]:
def parse(text):
    text = text.replace('\n', ' ', -1) + '\n\n'
    text = text.replace('\\tightlist', '', -1)
    return text

def parse_title(title):
    title = title.lower()
    title = title.replace(":", "-", -1)
    title = title.replace(" ", "-", -1)
    return title
    
# def parse_paragraph(pg):
#     text = pg.text
#     text = parse_html(pg, text)
#     return text

# def replace_span(span, text):
#     link = span.find('a')['href']
#     term = span.find('a').text
#     replacement = "\href{"+link+"}{"+term+"}"
#     return text.replace(term, replacement)

def parse_figure(fig, name):
    try: 
        link = fig.find('a')['href']
        img = requests.get(link, stream=True)
        img.raw.decode_content = True
        with open(name + link[-4:],'wb') as f:
            shutil.copyfileobj(img.raw, f)

        return "\includegraphics[width=0.9\linewidth]{"+name+"}"
    except:
        link = fig.find('div').next['src'].replace('_', '\_', -1)
        return f"Video: {link}"

def parse_header(header, rank):
    text = pypandoc.convert_text("<p>" + header.text + "</p>", 'latex', format='html')
    text = "section*{" + text + "}"
    if rank == "h3": text = "sub" + text
    elif rank != "h2": text = "subsub" + text
    return f"\\{text}"

# def parse_list(string, list_type):
#     name = "itemize"
#     if list_type == "ol": name = "enumerate"
    
def get_date(html_soup):
    date = html_soup.find('p', attrs={'class': 'MuiTypography-root MuiTypography-body1 MuiTypography-colorTextSecondary'}).text
    date = date[:date.find('\xa0')]
    if 'hour' in date:
        date = datetime.today().strftime("%b %-d, %Y")
    return date
    

def cleanup(fignum, temp):
    for i in range(fignum):
        for filename in glob.glob(f"./{i}*"):
            os.remove(filename) 
    for filename in glob.glob(f"./{temp}*"):
            os.remove(filename) 
    

In [67]:
texdoc = []
fignum = 0
temp = 'test'
title = html_soup.find('div', attrs={'class': 'MuiContainer-root MuiContainer-maxWidthLg'}).find('h1').text
author = html_soup.find('p', attrs={'class': 'MuiTypography-root MuiTypography-body1'}).find('a').text
texdoc.append("\documentclass{messari}\n")
texdoc.append("\\usepackage{hyperref}\n")
texdoc.append("\hypersetup{colorlinks=true,linkcolor=black,filecolor=magenta,urlcolor=blue,pdftitle={Overleaf Example},pdfpagemode=FullScreen}\n")
texdoc.append("\\title{" + title + "}\n")
texdoc.append("\\author{" + author + "}\n")
texdoc.append("\\date{" + get_date(html_soup) + "}\n")

texdoc.append("\\begin{document}\n")
texdoc.append("\\maketitle\n")
texdoc.append("\\author\n")



for i in range(len(main)):
    if main[i].name in ['p','ol', 'ul']:
        texdoc.append(parse(pypandoc.convert_text(str(main[i]), 'latex', format='html')))
    elif main[i].name == 'figure':
        texdoc.append(parse(parse_figure(main[i], str(fignum))))
        fignum += 1
    elif main[i].name in ['h2', 'h3', 'h4', 'h5', 'h6', 'h7']:
        texdoc.append(parse_header(main[i], main[i].name)+"\n")
        
texdoc.append("\end{document}")

with open(f"{temp}.tex", 'w') as fout:
    for i in range(len(texdoc)):
        fout.write(texdoc[i])
        
import os  
os.system(f"xelatex {temp}.tex")
os.rename(f"{temp}.pdf", f"{parse_title(title)}.pdf")
        
        
cleanup(fignum, temp)

This is XeTeX, Version 3.141592653-2.6-0.999993 (TeX Live 2021) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./test.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-02-18> (./messari.cls
Document Class: messari 2022/09/01 Notes
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2021/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/local/texlive/2021/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/local/texlive/2021/texmf-dist/tex/latex/fontspec/fontspec.sty
(/usr/local/texlive/2021/texmf-dist/tex/latex/l3packages/xparse/xparse.sty
(/usr/local/texlive/2021/texmf-dist/tex/latex/l3kernel/expl3.sty
(/usr/local/texlive/2021/texmf-dist/tex/latex/

In [83]:
driver.quit()

In [80]:
main = html_soup.find('div', attrs={'class': 'Messari-paywall'}).next('figure')

In [82]:
main[0]

<figure><a data-lightbox="true" href="https://messari.s3.amazonaws.com/images/agora-images/5E9F0678-Cosmos%20IBC.png"><img alt="" src="https://messari.s3.amazonaws.com/images/agora-images/5E9F0678-Cosmos%20IBC.png"/></a></figure>

In [350]:
output = pypandoc.convert_text(str(main[0]), 'latex', format='html')

In [351]:
output

'In recent weeks, the global fear of the rising interest rates and new\ncovid variant have generally slowed the price rise of "risk-on" assets,\nwith the majority of well-known DeFi tokens plunging by\n{\\href{https://twitter.com/Darrenlautf/status/1473067558588891138}{more\nthan 50\\%}}. The media focuses on price the most, but DeFi activities\nhave remained relatively unaffected. DeFi\'s total value locked (TVL) has\nremained stable, hovering at {\\href{https://defillama.com/}{\\$245\nbillion}}, or just around 6\\% below its all-time high.\n'

In [217]:
a = 'testing replace should be good'

In [218]:
a.replace('replace should', '1')

'testing 1 be good'

In [112]:
main.find('h1', attrs={'class': 'jss47'}).text

AttributeError: 'NoneType' object has no attribute 'find'

In [107]:
main.find('div', attrs={'class': 'jss54 jss56'})

<div class="jss54 jss56"><p>Since the start of the year, Terra’s native token LUNA has grown explosively, rising from a $300 million market cap to $6 billion. Was this run deserved?</p><p>In this report, I explore the factors driving demand for the Terra ecosystem including overviews of Chai, Mirror, and Anchor, as well as the big picture of what Terra is building.</p><figure><a data-lightbox="true" href="https://messari.s3.amazonaws.com/images/agora-images/95FCC89A-Jun%20Yu%20LUNA%20mcap.png"><img alt="" src="https://messari.s3.amazonaws.com/images/agora-images/95FCC89A-Jun%20Yu%20LUNA%20mcap.png"/></a></figure><p><strong><span>What’s driving this explosive growth?</span></strong></p><p>To understand why Terra’s ecosystem has been gaining so much traction recently, we must first understand the team and founding philosophy behind Terra. Terra’s co-founders Do Kwon and Daniel Shin both went through YCombinator which has incubated leading technology companies like Stripe, Airbnb, and Coi

In [72]:
elem = driver.find_element(By.XPATH, "//div[@class='jss33']")
elem = driver.find_element(By.XPATH, "//div[@class='jss54 jss56']")

In [76]:
elem.page_source

AttributeError: 'WebElement' object has no attribute 'page_source'

In [15]:
import urllib.request
response = urllib.request.urlopen(url)
html = response.read()

In [16]:
html

b'<!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no,maximum-scale=1"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="/manifest.json"/><link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png"/><link rel="shortcut icon" href="/favicon-96x96.png"/><link rel="alternate" type="application/rss+xml" title="Messari Crypto News Feed" href="https://messari.io/rss"/><title>Messari - Bitcoin & crypto price, news, charts, and research</title><meta name="google-site-verification" content="g9BAQ0QPz3z-nFOYtlwBMvlysVm5SPxCB6ghtHs0Wlw"/><meta name="google-site-verification" content="W2cUO8Zw2CD8KLXR-UP3NpJ3rE-vnUfHQkA5UQRxISE"/><meta name="description" content="Make smarter decisions. Messari gives you all the Bitcoin & crypto data, tools, and research you need." data-react-helmet="true"/><meta property="og:url" content="https://messari.io"/><meta property="og:title" con

In [20]:
url = 'https://messari.io/article/terra-the-defi-giant-reaching-mainstream-adoption'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [23]:
article = requests.get(url, headers=headers)

In [26]:
article.content

b'<!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no,maximum-scale=1"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="/manifest.json"/><link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png"/><link rel="shortcut icon" href="/favicon-96x96.png"/><link rel="alternate" type="application/rss+xml" title="Messari Crypto News Feed" href="https://messari.io/rss"/><title>Messari - Bitcoin & crypto price, news, charts, and research</title><meta name="google-site-verification" content="g9BAQ0QPz3z-nFOYtlwBMvlysVm5SPxCB6ghtHs0Wlw"/><meta name="google-site-verification" content="W2cUO8Zw2CD8KLXR-UP3NpJ3rE-vnUfHQkA5UQRxISE"/><meta name="description" content="Make smarter decisions. Messari gives you all the Bitcoin & crypto data, tools, and research you need." data-react-helmet="true"/><meta property="og:url" content="https://messari.io"/><meta property="og:title" con

In [4]:
a = 'tes.t.t..t.jpg'

In [5]:
a[a.rfind('.'):]

'.jpg'